In [1]:
####################################################################################################
########################### 1 - IMPORT DAS BIBLIOTECAS ############################################# 
####################################################################################################

import http.client
import json
import pandas as pd
from datetime import datetime
import numpy as np
import time

from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [2]:
####################################################################################################
#################################### 2 - FUNÇÕES ################################################### 
####################################################################################################

def datetime_handler(x):
    if isinstance(x, pd.Timestamp):
        return x.isoformat()
    raise TypeError("Unknown type")
    
def read_sheets(sheets):
    result = sheet.values().get(spreadsheetId='1P1LbVDyHmyebOUMIa-wyjyJ0fChiHh92p6Hj1FPPItE',
                            range=sheets).execute()
    values_old = result.get('values', [])
    return values_old


def insert_sheets(df, sheet):
    body = {
    'values': df
    }
    result = service.spreadsheets().values().update(
        spreadsheetId='1P1LbVDyHmyebOUMIa-wyjyJ0fChiHh92p6Hj1FPPItE', range= sheet,
        valueInputOption="USER_ENTERED", body=body).execute()

In [3]:
####################################################################################################
############################## 3 - API E SERVIÇOS GOOGLE ########################################### 
####################################################################################################

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()

In [ ]:
request1 = read_sheets('Request1!A2:A')
df_request1 = pd.DataFrame(data = request1)
df_request1 = df_request1.rename(columns = {0: 'idRequest'})


request2 = read_sheets('RequestCustomField!A2:A')
df_request2 = pd.DataFrame(data = request2)
df_request2 = df_request2.rename(columns = {0: 'idRequest'})

merge_request_comum = pd.merge(df_request1, df_request2, on = 'idRequest', how = 'inner')
merge_request1 = pd.concat([df_request1, merge_request_comum]).drop_duplicates(keep = False)
merge_request2 = pd.concat([df_request2, merge_request_comum]).drop_duplicates(keep = False)

merge_request = pd.concat([merge_request1, merge_request2]).drop_duplicates(keep = False)

merge_request = merge_request.values.tolist()

print(merge_request)



In [ ]:
####################################################################################################
################################## 5 - ETL E INSERT ################################################ 
####################################################################################################


#lista = list()
for x in range(0, len(merge_request)):
    try:
        time.sleep(0.2)
        conn = http.client.HTTPSConnection("api.pandape.com.br")
        payload = ''
        headers = {
          'Authorization': 'Bearer '
        }
        conn.request("GET", f"/v2/requests/{merge_request[x][0]}", payload, headers)
        res = conn.getresponse()
        data = res.read()

        data_dict = json.loads(data)

        data_json = json.dumps(data_dict['customFields'], default = datetime_handler)
        data_dict2 = json.loads(data_json)


        df_customField = pd.DataFrame(data = data_dict2)

        df_customField['idRequest'] = data_dict['idRequest']

        df_customField = df_customField[["idRequest", "idRequestField", "label", "value"]]

        df_customField = df_customField[df_customField['label'].isin(['Motivo da contratação',
                                                                      'Motivo da substituição',
                                                                      'Empresa',
                                                                      'Filial',
                                                                      'Departamento',
                                                                      'Área Operacional',
                                                                      'Perfil',
                                                                      'Gênero',
                                                                      'Turno',
                                                                      'Escala',
                                                                      'Horário de trabalho',
                                                                      'Nome do Posto',
                                                                      'Segmento'])]

        list_customField = df_customField.values.tolist()

        tamanho = len(read_sheets('RequestCustomField'))

        insert_sheets(list_customField, f'RequestCustomField!A{tamanho + 1}')
        
    except:
        pass
